In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

# Read CSV

In [ ]:
# Data source: https://afdc.energy.gov/vehicle-registration?year=2021
vehicle_df = pd.read_csv("/content/Vehicle_Registration_Counts_2021.csv",low_memory=False)
evreg_df = pd.read_csv("/content/EV_registration_counts.csv",low_memory=False)

print(vehicle_df.shape)
print(evreg_df.shape)

(52, 12)
(51, 2)


In [ ]:
evreg_df

,State,Registration Count
0,Alabama,"4,750"
1,Alaska,"1,290"
2,Arizona,"40,740"
3,Arkansas,"2,390"
4,California,"563,070"
5,Colorado,"37,000"
6,Connecticut,"13,350"
7,Delaware,"3,010"
8,District of Columbia,"3,700"
9,Florida,"95,640"


# Pre-processing

In [ ]:
# drop last row
vehicle_df=vehicle_df[vehicle_df['State']!='United States']

# drop Methanol column, all 0
vehicle_df.drop(columns='Methanol', inplace=True)

# rename column
vehicle_df.rename(columns={'State':'state'}, inplace=True)
vehicle_df.rename(columns={'Electric (EV)':'ev'}, inplace=True)
vehicle_df.rename(columns={'Plug-In Hybrid Electric (PHEV)':'phev'}, inplace=True)
vehicle_df.rename(columns={'Hybrid Electric (HEV)':'hev'}, inplace=True)
vehicle_df.rename(columns={'Biodiesel':'biodiesel'}, inplace=True)
vehicle_df.rename(columns={'Ethanol/Flex (E85)':'e85'}, inplace=True)
vehicle_df.rename(columns={'Compressed Natural Gas (CNG)':'cng'}, inplace=True)
vehicle_df.rename(columns={'Propane':'propane'}, inplace=True)
vehicle_df.rename(columns={'Hydrogen':'hydrogen'}, inplace=True)
vehicle_df.rename(columns={'Gasoline':'gasoline'}, inplace=True)
vehicle_df.rename(columns={'Diesel':'diesel'}, inplace=True)

evreg_df.rename(columns={'State':'state'}, inplace=True)
evreg_df.rename(columns={' Registration Count ':'registration_count'}, inplace=True)

# Transform to state code

In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    # "District of Columbia": "DC",
    # "American Samoa": "AS",
    # "Guam": "GU",
    # "Northern Mariana Islands": "MP",
    # "Puerto Rico": "PR",
    # "United States Minor Outlying Islands": "UM",
    # "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [ ]:
vehicle_df['state']=vehicle_df['state'].apply(lambda x : us_state_to_abbrev[x])
evreg_df['state']=evreg_df['state'].apply(lambda x : us_state_to_abbrev[x])


In [ ]:
# remove comma to avoid error when uploading to datbase(int type)
for col in ['ev','phev','hev','biodiesel','e85','cng','propane','hydrogen','gasoline','diesel']:
  vehicle_df[col]=vehicle_df[col].apply(lambda x: x.replace(",", ""))

evreg_df['registration_count']=evreg_df['registration_count'].apply(lambda x: x.replace(",", ""))

In [ ]:
vehicle_df

,state,ev,phev,hev,biodiesel,e85,cng,propane,hydrogen,gasoline,diesel
0,AL,4700,3300,42500,40500,449500,500,100,0,4051000,123500
1,AK,1300,500,7300,7600,50100,100,0,0,464200,31700
2,AZ,40700,15500,132200,51000,460400,900,900,0,5395300,191800
3,AR,2400,1800,26100,28700,290200,300,0,0,2241600,88800
4,CA,563100,315300,1355900,163600,1343200,12600,1500,11800,30512600,710500
5,CO,37000,16100,113600,53800,346700,600,100,0,4456600,208400
6,CT,13300,9200,55400,8800,140700,400,0,0,2578400,44300
7,DE,3000,2000,16700,4100,67400,100,0,0,796400,14600
8,DC,3700,2500,16100,300,17400,100,0,0,278900,2100
9,FL,95600,32200,287000,129300,1154600,600,100,0,15595900,336900


In [ ]:
evreg_df

,state,registration_count
0,AL,4750
1,AK,1290
2,AZ,40740
3,AR,2390
4,CA,563070
5,CO,37000
6,CT,13350
7,DE,3010
8,DC,3700
9,FL,95640


# Export CSV

In [ ]:
vehicle_df.to_csv("ldv_registration_counts_2021.csv", index=False)
evreg_df.to_csv("ev_registration_counts.csv", index=False)

The csv file is then inserted into database using DataGrip.